In [2]:
#This file reads the note_csv, spectrums. Generate and save the numpy arrays which are inputs to the model

In [3]:
import numpy as np
import glob

In [4]:
files = glob.glob('E:/notes_database/index/*')

In [5]:
def parse_row(csv_row):  # same as generate_database.ipynb

    csv_row_array = csv_row.split(",")

    filename = csv_row_array[0]
    time_array = [float(t) for t in csv_row_array[1::2]]
    note_array = [int(x) for x in csv_row_array[2::2]]

    assert len(time_array) == len(note_array)

#     print(time_array)
#     print(note_array)

    return filename, time_array, note_array

In [6]:
def parse_csv(csv_file): # Convert the csv files into numpy arrays
    
    with open(csv_file, 'r') as csvfile:
        csv_rows = csvfile.read().split("\n")

    filenames = []
    time_arrays = []
    note_arrays = []
    
    for csv_row in csv_rows:
        filename, time_array, note_array = parse_row(csv_row)
        
        filenames.append(filename)
        time_arrays.append(time_array)
        note_arrays.append(note_array)
    
    return filenames,time_arrays,note_arrays

In [7]:
def prepare_inputs(csv_name):
    # the time for a unit width of spectrum is about 0.003 second
    # however a unit in time_array is 0.5 second, so to calculate the unit numbers, should divide the time_number by 0.006
    filenames,time,note = parse_csv(csv_name)
    cnn_all_previous = []
    cnn_all_inputs = []
    y_all_train = []
    for file in range(len(filenames)):
        one_row = np.zeros(24)
        previous_time = [time[file][0]]
        spectrum = np.load(spectrums+filenames[file]+'.npy')
        y_train = []
        # load the spectrum numpy array from 20 units before the timing(onset) to 44 units after the timing
        for i in range(len(time[file])):
            
            if time[file][i] == previous_time[-1]:
                one_row[note[file][i]] = 1
            else:
                #append previous input (width:256)
                position = previous_time[-1]//0.006
                if position-20 < 256:
                    cnn_previous = np.zeros((108, 256))
                    cnn_previous[:, int(256-position+20):] = spectrum[:, :int(position-20)]
                else:
                    cnn_previous = spectrum[:, int(position-20-256):int(position-20)]
                cnn_all_previous.append(cnn_previous)
                
                #append y_train
                y_train.append(one_row)
                y_all_train.append(one_row)
                
                #append cnn input
                cnn_input = spectrum[:, int(previous_time[-1]//0.006-20):int(previous_time[-1]//0.006+44)]
                cnn_all_inputs.append(cnn_input)

                one_row = np.zeros(24)
                one_row[note[file][i]] = 1
                previous_time.append(time[file][i])
                
            if i == len(time[file]) - 1:
                #append previous input (width:256)
                position = previous_time[-1]//0.006
                if position-20 < 256:
                    cnn_previous = np.zeros((108, 256))
                    cnn_previous[:, int(256-position+20):] = spectrum[:, :int(position-20)]
                else:
                    cnn_previous = spectrum[:, int(position-20-256):int(position-20)]
                cnn_all_previous.append(cnn_previous)
                
                #append y_train
                y_train.append(one_row)
                y_all_train.append(one_row)
                
                #append cnn input
                cnn_input = spectrum[:, int(previous_time[-1]//0.006-20):int(previous_time[-1]//0.006+44)]
                cnn_all_inputs.append(cnn_input)
                
        #check if the last cnn_input has right shape
        if cnn_all_inputs[-1].shape != (108, 64):
            cnn_all_inputs = cnn_all_inputs[:-1]
            cnn_all_previous = cnn_all_previous[:-1]
            y_all_train = y_all_train[:-1]
            
    cnn_all_previous = np.array(cnn_all_previous).reshape(len(cnn_all_previous), 108, 256, 1)
    cnn_all_inputs = np.array(cnn_all_inputs).reshape(len(cnn_all_inputs), 108, 64, 1)
    y_all_train = np.array(y_all_train)
    return cnn_all_previous, cnn_all_inputs, y_all_train

In [8]:
save = 'E:/notes_database/'

In [9]:
spectrums = 'E:/notes_database/spectrograms/'

In [10]:
count = 0
number = 0
prev_save, cnn_save, y_train_save = prepare_inputs(files[0])
for i in range(len(files))[1:]:
    if number < 5:
        cnn_prev, cnn, y_train = prepare_inputs(files[i])
        prev_save = np.append(prev_save, cnn_prev, axis=0)
        cnn_save = np.append(cnn_save, cnn, axis=0)
        y_train_save = np.append(y_train_save, y_train, axis=0)
        number += 1
    elif number >= 5:
        np.save(save+'CNN_previous/{}.npy'.format(count), prev_save.astype(np.float16))
        np.save(save+'pureCNN/{}.npy'.format(count), cnn_save.astype(np.float16))
        np.save(save+'y_pureCNN/{}.npy'.format(count), y_train_save.astype(np.float16))
        count += 1
        number = 1
        prev_save, cnn_save, y_train_save = prepare_inputs(files[i])
        
np.save(save+'CNN_previous/{}.npy'.format(count), prev_save.astype(np.float16))
np.save(save+'pureCNN/{}.npy'.format(count), cnn_save.astype(np.float16))
np.save(save+'y_pureCNN/{}.npy'.format(count), y_train_save.astype(np.float16))